In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

Primero codificados y estandarizados:

In [2]:
df=pd.read_csv('data/bikes_regis_estand_encod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,registrados,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.295131,-0.376545,654,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,-0.722312,-0.740034,0.507407,0.882110,670,2018-01-02,NaN,0,1,1,1,0,0
2,3,1,-1.642047,-1.746977,-1.354487,0.861132,1229,2018-01-03,NaN,0,1,2,1,0,0


In [3]:
df.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'registrados', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [4]:
X=df.drop(['registrados'],axis=1)
y=df['registrados']

In [5]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
param = {"max_depth": [5, 7, 8], 
         "max_features": [1,2,3], 
         "min_samples_split": [10, 75, 150], 
         "min_samples_leaf": [10, 75, 150]}

In [8]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [9]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 7, 8], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 75, 150],
                         'min_samples_split': [10, 75, 150]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [10]:

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=8, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [12]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [13]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [29]:
rf_encod_estand_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest registrados cod - estand")
rf_encod_estand_results

,MAE,MSE,RMSE,R2,set,modelo
0,480.063816,370948.974188,609.055805,0.828151,test,Random Forest registrados cod - estand
1,483.013397,410068.605164,640.365993,0.835517,train,Random Forest registrados cod - estand


In [15]:
importancia_predictores = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': bosque.feature_importances_}
                            )


# ordenamos de mayor a menor los resultados
importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
9,año_map,0.340827
1,temperatura,0.187575
2,sens_termica,0.158115
10,estacion_map,0.108778
6,mes_bueno,0.071145
7,dia_semana_nuevo,0.045588
3,humedad,0.041274
4,viento,0.028306
0,clima,0.015210
8,no_laboral_nuevo,0.003182


## Ahora sin estandarizar:

In [16]:
df2 = pd.read_csv('data/bikes_regis_encod.csv', index_col=0)

In [17]:
df2.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df2.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'registrados', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [19]:
X2=df2.drop(['registrados'],axis=1)
y2=df2['registrados']

In [20]:
# Separamos en train y test
x2_train, x2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)


In [21]:
param2 = {"max_depth": [5, 7, 8], 
         "max_features": [1,2,3], 
         "min_samples_split": [10, 75, 150], 
         "min_samples_leaf": [10, 75, 150]}

In [22]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [23]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf2.fit(x2_train, y2_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 7, 8], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 75, 150],
                         'min_samples_split': [10, 75, 150]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [24]:
bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestRegressor(max_depth=8, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [25]:
y2_pred_test_rf = bosque2.predict(x2_test)
y2_pred_train_rf = bosque2.predict(x2_train)

In [26]:
rf_resgistrados_encod = metricas(y2_test, y2_train, y2_pred_test_rf, y2_pred_train_rf, "Random Forest registrados codificados")
rf_resgistrados_encod 

,MAE,MSE,RMSE,R2,set,modelo
0,511.700508,425375.920176,652.208494,0.802936,test,Random Forest registrados codificados
1,501.242424,444380.888630,666.618998,0.821754,train,Random Forest registrados codificados


In [27]:

# vamos a crearnos un dataframe, igual que hicimos en la clase anterior con la importancia de cada una de las variables incluidas en el modelo

importancia_predictores2 = pd.DataFrame(
                            {'predictor': x2_train.columns,
                             'importancia': bosque2.feature_importances_}
                            )


# ordenamos de mayor a menor los resultados
importancia_predictores2.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores2

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
9,año_map,0.309244
2,sens_termica,0.204944
1,temperatura,0.173563
10,estacion_map,0.097168
6,mes_bueno,0.082075
3,humedad,0.043119
7,dia_semana_nuevo,0.041769
4,viento,0.028409
0,clima,0.016761
8,no_laboral_nuevo,0.002947
